<a href="https://colab.research.google.com/github/labadilloj/TLG-DS-2025/blob/main/DS_C2_SC2_LESLIE_BADILLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Análisis del progreso mundial de vacunación##


In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()  # seleccionar archivo .csv


Saving country_vaccinations.csv to country_vaccinations (3).csv


a. Extraer la información del archivo.

In [ ]:
df = pd.read_csv("country_vaccinations.csv")
print(df.head())

       country iso_code        date  total_vaccinations  people_vaccinated  \
0  Afghanistan      AFG  2021-02-22                 0.0                0.0   
1  Afghanistan      AFG  2021-02-23                 NaN                NaN   
2  Afghanistan      AFG  2021-02-24                 NaN                NaN   
3  Afghanistan      AFG  2021-02-25                 NaN                NaN   
4  Afghanistan      AFG  2021-02-26                 NaN                NaN   

   people_fully_vaccinated  daily_vaccinations_raw  daily_vaccinations  \
0                      NaN                     NaN                 NaN   
1                      NaN                     NaN              1367.0   
2                      NaN                     NaN              1367.0   
3                      NaN                     NaN              1367.0   
4                      NaN                     NaN              1367.0   

   total_vaccinations_per_hundred  people_vaccinated_per_hundred  \
0                 

b. Mostrar la estructura y tipos de datos de cada columna para identificar qué operaciones puedes realizar con cada una de ellas, asegurándote que las columnas con fechas sean del tipo datetime64.

In [ ]:
print(df.dtypes) #tipo de datp
df["date"] = pd.to_datetime(df["date"], errors="coerce") #cambbio de data
print(df["date"].dtype)   # debería salir datetime64[ns]
print(df["date"])

country                                 object
iso_code                                object
date                                    object
total_vaccinations                     float64
people_vaccinated                      float64
people_fully_vaccinated                float64
daily_vaccinations_raw                 float64
daily_vaccinations                     float64
total_vaccinations_per_hundred         float64
people_vaccinated_per_hundred          float64
people_fully_vaccinated_per_hundred    float64
daily_vaccinations_per_million         float64
vaccines                                object
source_name                             object
source_website                          object
dtype: object
datetime64[ns]
0       2021-02-22
1       2021-02-23
2       2021-02-24
3       2021-02-25
4       2021-02-26
           ...    
86507   2022-03-25
86508   2022-03-26
86509   2022-03-27
86510   2022-03-28
86511   2022-03-29
Name: date, Length: 86512, dtype: datetime64[ns]


c. Determinar la cantidad de vacunas aplicadas de cada compañía (con base en cómo lo reporta cada país en la columna vaccines, en otras palabras, agrupe por vaccines y realice la sumatoria).

In [ ]:
resumen_vaccines = (
    df.groupby('vaccines', as_index=False)['daily_vaccinations']
      .sum()
      .sort_values('daily_vaccinations', ascending=False)
)

resumen_vaccines['daily_vaccinations'] = (
    resumen_vaccines['daily_vaccinations'].round(0).astype('int64')
)

print("\nCantidad de vacunas por compañía:")
print(resumen_vaccines.head(25))  # muestra las primeras 20 filas


Cantidad de vacunas por compañía:
                                             vaccines  daily_vaccinations
10  CanSino, Sinopharm/Beijing, Sinopharm/Wuhan, S...          3250342496
16             Covaxin, Oxford/AstraZeneca, Sputnik V          1829085487
50       Moderna, Oxford/AstraZeneca, Pfizer/BioNTech           620164884
33          Johnson&Johnson, Moderna, Pfizer/BioNTech           589020809
20  Johnson&Johnson, Moderna, Novavax, Oxford/Astr...           505216912
24  Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...           485378897
41  Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...           414643555
21  Johnson&Johnson, Moderna, Novavax, Oxford/Astr...           373213575
26  Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...           232845439
5   CanSino, Covaxin, Moderna, Oxford/AstraZeneca,...           217745032
1   Abdala, Moderna, Oxford/AstraZeneca, Pfizer/Bi...           201816053
6   CanSino, Johnson&Johnson, Moderna, Oxford/Astr...           190175636
18 

d. Obtener la cantidad de vacunas aplicadas en todo el mundo.

In [ ]:
total_global_daily = df['daily_vaccinations'].sum()
print(f"Total de vacunas: {int(total_global_daily):,}")

Total de vacunas: 11,320,239,871


e. Calcular el promedio de vacunas aplicadas por país.

In [ ]:
promedio_por_pais = (
    df.groupby('country', as_index=False)['daily_vaccinations']
      .mean()                                  # promedio diario
      .rename(columns={'daily_vaccinations':'promedio_diario'})
      .sort_values('promedio_diario', ascending=False)
)

promedio_por_pais['promedio_diario'] = (
    promedio_por_pais['promedio_diario'].round(0).astype('int64')
)

print("Promedio diario de vacunas aplicadas por país:")
print(promedio_por_pais.head(20))  # muestra los 20 mayores


Promedio diario de vacunas aplicadas por país:
            country  promedio_diario
41            China          6930368
91            India          4175994
212   United States          1191727
27           Brazil           943529
92        Indonesia           846289
100           Japan           621580
15       Bangladesh           545305
151        Pakistan           543005
217         Vietnam           531095
127          Mexico           413425
75          Germany           376158
157     Philippines           366566
93             Iran           353519
163          Russia           348084
204          Turkey           335192
197        Thailand           325147
211  United Kingdom           314084
70           France           310496
185     South Korea           304251
98            Italy           297058


f. Determinar la cantidad de vacunas aplicadas el día 29/01/21 en todo el mundo.

In [ ]:
#print(df['date'])
f = pd.Timestamp("2021-01-29")
filtro_fecha = df.loc[df['date'] == f].copy()

total_dia = pd.to_numeric(filtro_fecha['daily_vaccinations']).sum()
print(f"Total de vacunas aplicadas el {f.date()}: {int(total_dia):,}")

Total de vacunas aplicadas el 2021-01-29: 4,884,052


g. Crear un dataframe nuevo denominado conDiferencias que contenga los datos originales y una columna derivada (diferencias) con las diferencias de aplicación entre las columnas daily_vaccionations y daily_vaccionations_raw.  

In [ ]:
conDiferencias = df.copy()          # copia del original
conDiferencias['diferencias'] = conDiferencias['daily_vaccinations'] - conDiferencias['daily_vaccinations_raw']
print(conDiferencias[['daily_vaccinations', 'daily_vaccinations_raw', 'diferencias']])
#print(conDiferencias.head())  #Revisión de data completa

       daily_vaccinations  daily_vaccinations_raw  diferencias
0                     NaN                     NaN          NaN
1                  1367.0                     NaN          NaN
2                  1367.0                     NaN          NaN
3                  1367.0                     NaN          NaN
4                  1367.0                     NaN          NaN
...                   ...                     ...          ...
86507             69579.0                139213.0     -69634.0
86508             83429.0                100086.0     -16657.0
86509             90629.0                 53311.0      37318.0
86510            100614.0                 89321.0      11293.0
86511            103751.0                105369.0      -1618.0

[86512 rows x 3 columns]


h. Obtener el periodo de tiempo entre el registro con fecha más reciente y el registro con fecha más antigua.

In [ ]:
primera_fecha = df['date'].min()
ultima_fecha   = df['date'].max()

print("Más antigua:", primera_fecha)
print("Más reciente:", ultima_fecha)

rango = ultima_fecha - primera_fecha
print("Días:", rango.days)

Más antigua: 2020-12-02 00:00:00
Más reciente: 2022-03-29 00:00:00
Días: 482


i. Crear un dataframe nuevo denominado conCantidad que contenga los datos originales y una columna derivada (canVac) con la cantidad de vacunas utilizadas cada día (usar la columna vaccines y separar por el carácter , ).

In [ ]:
conCantidad = df.copy()          # copia del original
conCantidad['canVac'] = conCantidad['vaccines'].str.split(', ').str.len()
print(conCantidad[['vaccines', 'canVac']])
#print(conCantidad.head())  #Revisión de data completa

                                                vaccines  canVac
0      Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...       4
1      Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...       4
2      Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...       4
3      Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...       4
4      Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...       4
...                                                  ...     ...
86507  Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...       4
86508  Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...       4
86509  Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...       4
86510  Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...       4
86511  Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...       4

[86512 rows x 2 columns]


j. Generar un dataframe denominado antes20 con todos los registros que se hayan realizado antes del 20 de diciembre de 2020.

In [ ]:
antes20 = df.copy()          # copia del original
antes20 = antes20[antes20['date'] < '2020-12-20']
print(antes20)


             country iso_code       date  total_vaccinations  \
13403         Canada      CAN 2020-12-14                 5.0   
13404         Canada      CAN 2020-12-15               727.0   
13405         Canada      CAN 2020-12-16              3025.0   
13406         Canada      CAN 2020-12-17              7279.0   
13407         Canada      CAN 2020-12-18             11296.0   
...              ...      ...        ...                 ...   
82362  United States      USA 2020-12-15             84638.0   
82363  United States      USA 2020-12-16            244549.0   
82364  United States      USA 2020-12-17            517161.0   
82365  United States      USA 2020-12-18            933551.0   
82366  United States      USA 2020-12-19           1115437.0   

       people_vaccinated  people_fully_vaccinated  daily_vaccinations_raw  \
13403                5.0                      NaN                     NaN   
13404              727.0                      NaN                   722.0   


k. Obtener un dataframe denominado pfizer con todos los registros donde se haya utilizado la vacuna Pfizer.

In [ ]:
pfizer = df.copy()        # copia del original
col = 'tipo_vacuna' if 'tipo_vacuna' in df.columns else 'vaccines'

pfizer = df.loc[
    df[col].str.contains('Pfizer/BioNTech', case=False, na=False, regex=False)
].copy()

print(pfizer.head())
total_filas = len(pfizer)
print("Total de filas:", total_filas)


       country iso_code       date  total_vaccinations  people_vaccinated  \
0  Afghanistan      AFG 2021-02-22                 0.0                0.0   
1  Afghanistan      AFG 2021-02-23                 NaN                NaN   
2  Afghanistan      AFG 2021-02-24                 NaN                NaN   
3  Afghanistan      AFG 2021-02-25                 NaN                NaN   
4  Afghanistan      AFG 2021-02-26                 NaN                NaN   

   people_fully_vaccinated  daily_vaccinations_raw  daily_vaccinations  \
0                      NaN                     NaN                 NaN   
1                      NaN                     NaN              1367.0   
2                      NaN                     NaN              1367.0   
3                      NaN                     NaN              1367.0   
4                      NaN                     NaN              1367.0   

   total_vaccinations_per_hundred  people_vaccinated_per_hundred  \
0                       

l. Almacenar los dataframes generados (conDiferencias, conCantidad, antes20 y pfizer) en un archivo de Excel denominado resultadosReto.xlsx, donde cada dataframe ocupe una hoja diferente. Se recomienda ver la documentación de pd.ExcelWriter.

In [ ]:
# Guarda cada DataFrame en una hoja distinta del mismo archivo Excel
with pd.ExcelWriter("resultadosReto.xlsx", engine="openpyxl") as writer:
    conDiferencias.to_excel(writer, sheet_name="conDiferencias", index=False)
    conCantidad.to_excel(writer,    sheet_name="conCantidad",   index=False)
    antes20.to_excel(writer,        sheet_name="antes20",       index=False)
    pfizer.to_excel(writer,         sheet_name="pfizer",        index=False)

print("Archivo 'resultadosReto.xlsx' creado con 4 hojas")

Archivo 'resultadosReto.xlsx' creado con 4 hojas
